In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from utils.regexp_validation import validate_postag_regexp, validate_token_regexp

# simple testing


In [3]:
fp_postag = """
<rule id="BRIEFCATCH_23860985463569237141502738364142673900" name="BRIEFCATCH_PUNCHINESS_892">
    <pattern>
        <token postag_regexp="yes" postag="VBG">
        </token>
    </pattern>
</rule>
"""
print(validate_postag_regexp(fp_postag))


<rule id="BRIEFCATCH_23860985463569237141502738364142673900" name="BRIEFCATCH_PUNCHINESS_892">
    <pattern>
        <token postag="VBG">
        </token>
    </pattern>
</rule>



In [4]:
fn_postag = """
<rule id="BRIEFCATCH_23860985463569237141502738364142673900" name="BRIEFCATCH_PUNCHINESS_892">
    <pattern>
        <token postag="VBG|V.*"></token>
    </pattern>
</rule>
"""
print(validate_postag_regexp(fn_postag))


<rule id="BRIEFCATCH_23860985463569237141502738364142673900" name="BRIEFCATCH_PUNCHINESS_892">
    <pattern>
        <token postag_regexp="yes" postag="VBG|V.*"></token>
    </pattern>
</rule>



In [5]:
token_fn = """
</rule>
<rule id="BRIEFCATCH_142810940769371792650075747430090357151" name="BRIEFCATCH_PUNCHINESS_1155">
    <antipattern>
        <token>material|substantial</token>
    </antipattern>
</rule>
"""
print(validate_token_regexp(token_fn))


</rule>
<rule id="BRIEFCATCH_142810940769371792650075747430090357151" name="BRIEFCATCH_PUNCHINESS_1155">
    <antipattern>
        <token regexp="yes">material|substantial</token>
    </antipattern>
</rule>



In [6]:
token_fp = """
</rule>
<rule id="BRIEFCATCH_142810940769371792650075747430090357151" name="BRIEFCATCH_PUNCHINESS_1155">
    <antipattern>
        <token regexp="yes">material</token>
    </antipattern>
</rule>
"""
print(validate_token_regexp(token_fp))


</rule>
<rule id="BRIEFCATCH_142810940769371792650075747430090357151" name="BRIEFCATCH_PUNCHINESS_1155">
    <antipattern>
        <token>material</token>
    </antipattern>
</rule>



# full rule testing


In [7]:
# 5th token postag should not have postag_regexp="yes"
r_postag_false_positive = """
<rule id="BRIEFCATCH_23860985463569237141502738364142673900" name="BRIEFCATCH_PUNCHINESS_892">
    <pattern>
        <token inflected="yes">have</token>
        <token min="0" regexp="yes">some|real|great|considerable|much</token>
        <token>difficulty</token>
        <token min="0">in</token>
        <token postag_regexp="yes" postag="VBG">
            <exception regexp="yes">accepting|concerning|following|maintaining|obtaining|pending|regarding|seeing|standing</exception>
        </token>
    </pattern>
    <message>Would shorter words add punch?|**Example** from Justice Thomas: “Only later did courts **wrestle with** the language in § 230(f )(3) suggesting providers are liable for content they help develop ‘in part.’”|**Example** from Justice Barrett: “We need not **wrestle with** these questions in this case, though, because the same standard applies regardless whether *Smith* stays or goes.”</message>
    <suggestion>\1 trouble \5</suggestion>
    <suggestion><match no="1" postag="(V.*)" postag_regexp="yes" postag_replace="$1">struggle</match> to <match no="5" postag="V.*" postag_regexp="yes" postag_replace="VB"/></suggestion>
    <suggestion><match no="1" postag="(V.*)" postag_regexp="yes" postag_replace="$1">wrestle</match> with \5</suggestion>
    <short>{"ruleGroup":"BRIEFCATCH_PUNCHINESS_892","ruleGroupIdx":0,"isConsistency":false,"isStyle":true,"correctionCount":3,"priority":"5.543"}</short>
    <example correction="have trouble reaching|struggle to reach|wrestle with reaching">The experts can <marker>have difficulty reaching</marker> consensus.</example>
</rule>
"""
# this should remove postag_regexp from fifth token
print(validate_postag_regexp(r_postag_false_positive))


<rule id="BRIEFCATCH_23860985463569237141502738364142673900" name="BRIEFCATCH_PUNCHINESS_892">
    <pattern>
        <token inflected="yes">have</token>
        <token min="0" regexp="yes">some|real|great|considerable|much</token>
        <token>difficulty</token>
        <token min="0">in</token>
        <token postag="VBG">
            <exception regexp="yes">accepting|concerning|following|maintaining|obtaining|pending|regarding|seeing|standing</exception>
        </token>
    </pattern>
    <message>Would shorter words add punch?|**Example** from Justice Thomas: “Only later did courts **wrestle with** the language in § 230(f )(3) suggesting providers are liable for content they help develop ‘in part.’”|**Example** from Justice Barrett: “We need not **wrestle with** these questions in this case, though, because the same standard applies regardless whether *Smith* stays or goes.”</message>
    <suggestion> trouble </suggestion>
    <suggestion><match no="1" postag="(V.*)" postag_re

In [8]:
# modified verison of above rule (see postag of 5th token)
r_postag_false_negative = """
<rule id="BRIEFCATCH_23860985463569237141502738364142673900" name="BRIEFCATCH_PUNCHINESS_892">
    <pattern>
        <token inflected="yes">have</token>
        <token min="0" regexp="yes">some|real|great|considerable|much</token>
        <token>difficulty</token>
        <token min="0">in</token>
        <token postag="VBG|V.*">
            <exception regexp="yes">accepting|concerning|following|maintaining|obtaining|pending|regarding|seeing|standing</exception>
        </token>
    </pattern>
</rule>
"""
# should add postag_regexp to fifth token
print(validate_postag_regexp(r_postag_false_negative))


<rule id="BRIEFCATCH_23860985463569237141502738364142673900" name="BRIEFCATCH_PUNCHINESS_892">
    <pattern>
        <token inflected="yes">have</token>
        <token min="0" regexp="yes">some|real|great|considerable|much</token>
        <token>difficulty</token>
        <token min="0">in</token>
        <token postag_regexp="yes" postag="VBG|V.*">
            <exception regexp="yes">accepting|concerning|following|maintaining|obtaining|pending|regarding|seeing|standing</exception>
        </token>
    </pattern>
</rule>



In [9]:
# first token missing regexp='yes'
r_token_false_negative = """
</rule>
<rule id="BRIEFCATCH_142810940769371792650075747430090357151" name="BRIEFCATCH_PUNCHINESS_1155">
    <antipattern>
        <token inflected="yes">constitute|amount|result|lead</token>
        <token regexp="yes">to|in</token>
        <token regexp="yes">a|an</token>
        <token regexp="yes">absolute|actual|appreciable|clear|complete|considerable|credible|definitive|deliberate|bold|distinct|explicit|express|full|genuine|patent|real|realistic|robust|serious|solid|strong|substantial|tangible|total|true|viable</token>
        <token regexp="yes">material|substantial</token>
        <token>breach</token>
        <token>of</token>
        <token regexp="yes">a|an|article|contract|duty|his|its|their|trust</token>
    </antipattern>
    <pattern>
        <marker>
            <token inflected="yes" regexp="yes">constitute|amount|result|lead</token>
            <token min="0" regexp="yes">to|in</token>
            <token regexp="yes">a|an</token>
            <token min="0" regexp="yes">absolute|actual|appreciable|clear|complete|considerable|credible|definitive|deliberate|bold|distinct|explicit|express|full|genuine|patent|real|realistic|robust|serious|solid|strong|substantial|tangible|total|true|viable</token>
            <token min="0" regexp="yes">material|substantial</token>
            <token>breach</token>
            <token>of</token>
            <token min="0">the</token>
        </marker>
        <token>
            <exception>the</exception>
        </token>
    </pattern>
    <message>Would a stronger verb help engage the reader?|**Example** from Justice Kagan: "Nebraska did not purposefully set out to **breach** the Compact."|**Example** from Morgan Chu: "[Plaintiff] is reluctant to **breach** its agreement for a simple reason . . . ."</message>
    <suggestion><match no="5" regexp_match="^(.+)$" regexp_replace="$1ly"/> <match no="1" postag="(V.*)" postag_regexp="yes" postag_replace="$1">breach</match> the</suggestion>
    <suggestion><match no="5" regexp_match="^(.+)$" regexp_replace="$1ly"/> <match no="1" postag="(V.*)" postag_regexp="yes" postag_replace="$1">breach</match></suggestion>
    <short>{"ruleGroup":"BRIEFCATCH_PUNCHINESS_1155","ruleGroupIdx":1,"isConsistency":false,"isStyle":true,"correctionCount":2,"priority":"8.832"}</short>
    <example correction="breached the|breached">This <marker>amounted to a clear breach of the</marker> contract.</example>
    <example>This constituted an actual breach of their trust.</example>
</rule>
"""
# should add regexp to first token
print(validate_token_regexp(r_token_false_negative))


</rule>
<rule id="BRIEFCATCH_142810940769371792650075747430090357151" name="BRIEFCATCH_PUNCHINESS_1155">
    <antipattern>
        <token regexp="yes" inflected="yes">constitute|amount|result|lead</token>
        <token regexp="yes">to|in</token>
        <token regexp="yes">a|an</token>
        <token regexp="yes">absolute|actual|appreciable|clear|complete|considerable|credible|definitive|deliberate|bold|distinct|explicit|express|full|genuine|patent|real|realistic|robust|serious|solid|strong|substantial|tangible|total|true|viable</token>
        <token regexp="yes">material|substantial</token>
        <token regexp="yes">breach</token>
        <token regexp="yes">of</token>
        <token regexp="yes">a|an|article|contract|duty|his|its|their|trust</token>
    </antipattern>
    <pattern>
        <marker>
            <token inflected="yes" regexp="yes">constitute|amount|result|lead</token>
            <token min="0" regexp="yes">to|in</token>
            <token regexp="yes">a|an</token

In [10]:
# modified first token from rule above
r_token_false_positive = """
</rule>
<rule id="BRIEFCATCH_142810940769371792650075747430090357151" name="BRIEFCATCH_PUNCHINESS_1155">
    <antipattern>
        <token regexp="yes">to</token>
        <token regexp="yes">a|an</token>
        <token regexp="yes">absolute|actual|appreciable|clear|complete|considerable|credible|definitive|deliberate|bold|distinct|explicit|express|full|genuine|patent|real|realistic|robust|serious|solid|strong|substantial|tangible|total|true|viable</token>
        <token regexp="yes">material|substantial</token>
        <token>breach</token>
        <token>of</token>
        <token regexp="yes">a|an|article|contract|duty|his|its|their|trust</token>
    </antipattern>
</rule>
"""
# should remove regexp from first token
print(validate_token_regexp(r_token_false_positive))


</rule>
<rule id="BRIEFCATCH_142810940769371792650075747430090357151" name="BRIEFCATCH_PUNCHINESS_1155">
    <<token>a|an|article|contract|duty|his|its|their|trust</token>tipattern>
        <token>to</token>
        <token>a|<token>a|an|article|contract|duty|his|its|their|trust</token></token>|<token>a|<token>a|an|article|contract|duty|his|its|their|trust</token></token>
        <token>a|<token>a|an|article|contract|duty|his|its|their|trust</token></token>bsolute|<token>absolute|actual|appreciable|clear|complete|considerable|credible|definitive|deliberate|bold|distinct|explicit|express|full|genuine|patent|real|realistic|robust|serious|solid|strong|subst<token>a|an|article|contract|duty|his|its|their|trust</token>tial|t<token>a|an|article|contract|duty|his|its|their|trust</token>gible|total|true|viable</token>|<token>absolute|actual|appreciable|clear|complete|considerable|credible|definitive|deliberate|bold|distinct|explicit|express|full|genuine|patent|real|realistic|robust|serious|sol